In [1]:
pip install load_dotenv

Note: you may need to restart the kernel to use updated packages.


In [46]:
from crewai import Agent ,LLM
from dotenv import load_dotenv
from crewai_tools import SerperDevTool
# from tools import tool
import os


C:\Users\kusha\anaconda3\Lib\site-packages\paramiko\pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\kusha\anaconda3\Lib\site-packages\paramiko\transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [48]:
# Load environment variables
load_dotenv()


True

In [50]:
llm = LLM(
    model="gpt-4",
    temperature=0.7,
    api_key=os.getenv("OPENAI_API_KEY")
)

In [52]:
# Fetch and set SERPER_API_KEY in environment if available
serper_api_key = os.getenv("SERPER_API_KEY")
if serper_api_key is None:
    raise ValueError("SERPER_API_KEY is not set. Please add it to your .env file.")
else:
    os.environ["SERPER_API_KEY"] = serper_api_key

# Instantiate the Serper dev tool
tool = SerperDevTool()

generator = Agent(
    role="Passage generator and question framer",
    goal="Generate a passage for {grade} students of difficulty level {level} "
         "and frame {question_nos} multiple-choice questions respectively.",
    verbose=True,
    memory=True,
    backstory=(
        "You are responsible for creating knowledge-enhancing passages and questions "
        "suited for the grade level and difficulty specified by the user."
    ),
    tools=[tool],
    llm=llm,
    allow_delegation=True
)

# Create the evaluator agent
evaluator = Agent(
    role="Answer evaluator and scorer",
    goal="Using the passage and questions generated, determine the correct answers and "
         "evaluate the student's answers. Provide a score and feedback based on performance.",
    verbose=True,
    memory=True,
    backstory=(
        "You are an evaluator who assesses student answers based on the passage and questions generated, "
        "automatically determining correct answers for scoring."
    ),
    tools=[tool],
    llm=llm,
    allow_delegation=True
)


In [79]:
from pydantic import BaseModel
from typing import List

# Define the Pydantic models
class QuestionModel(BaseModel):
    question: str
    options: List[str]
    answer: str

class QuizOutput(BaseModel):
    passage: str
    questions: List[QuestionModel]
generator_task = Task(
    description=(
        "The generator agent is responsible for creating a descriptive educational passage of minimum 1 page for a specified grade level. "
        "This passage should be tailored to a difficulty level that aligns with the student's abilities. "
        "After generating the passage, the generator agent must also frame multiple-choice questions "
        "based on the passage content. These questions should be structured to assess the student's comprehension "
        "and critical thinking, with four answer choices per question. The agent must ensure that the passage "
        "and questions are contextually and grammatically accurate for the educational setting."
    ),
    expected_output=(
        "A long descriptive passage suitable for {grade} students, tailored to the {level} difficulty level. "
        "This passage includes {question_nos} multiple-choice questions, each with four answer options on the topic {topic} as specified. "
        "The passage and questions should be clear, educational, and aligned with the specified grade and "
        "difficulty level to ensure relevance and appropriateness."
    ),
    tools=[tool],  # Tools used (make sure this is defined elsewhere)
    agent=generator,  # The agent executing the task
    output_pydantic=QuizOutput,  # Correct output model type
)

# Describing the Task of the evaluator agent
evaluator_task = Task(
    description=(
        "The evaluator agent's task is to assess the student's responses to the generated questions. "
        "This involves analyzing each student-provided answer against the passage content and determining "
        "the correct answers. The evaluator agent will then assign a score based on the number of correct answers "
        "and provide personalized feedback for each question, highlighting areas of strength and areas that may need "
        "improvement. The feedback should be constructive, with a focus on encouraging further learning and development."
    ),
    expected_output=(
        "An evaluation report including the student's score as a percentage, along with detailed feedback "
        "for each question. The feedback should include whether each answer was correct or incorrect, along with "
        "explanations where necessary. The evaluation should offer a balanced view of the student's performance, "
        "recognizing strengths and identifying areas for improvement."
    ),
    tools=[tool],  # Tools used (make sure this is defined elsewhere)
    agent=evaluator,  # The agent executing the task
)

In [81]:
from crewai import Crew, Process
import json

main_crew = Crew(
    agents=[generator],
    tasks=[generator_task],
    process=Process.sequential,
)

2024-12-19 00:24:30,630 - 29256 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [83]:

# Collect input from the user
grade = input("Enter the grade: ").strip()
question_nos = int(input("Enter the number of questions: ").strip())
level = input("Enter the level of the questions: ").strip()
topic = input("Enter the topic: ").strip()

# Validate inputs
if not grade or question_nos <= 0 or not level or not topic:
    raise ValueError("Invalid input values provided.")

try:
    # Running the crew's task
    result = main_crew.kickoff(inputs={
        "grade": grade,
        "question_nos": question_nos,
        "level": level,
        "topic": topic
    })

    # Checking if result is not None
    if result.json_dict:
        # Print result for debugging
        print("This is the result: ", result.json_dict)
        
        # Extract JSON output
        json_output = result.json_dict  # Use json_dict to get the actual Python dictionary
        with open("questions.json", "w") as file:
            json.dump(json_output, file, indent=4)

        # Display the passage and questions (without answers) to students
        print("\nGenerated Passage:")
        print(json_output['passage'])
        
        print("\nGenerated Questions:")
        for i, question in enumerate(json_output['questions'], start=1):
            print(f"\nQuestion {i}: {question['question']}")
            for j, option in enumerate(question['options'], start=1):
                print(f"  {j}. {option}")
    else:
        print("No result output available.")
        
except ValueError as ve:
    print("Error:", ve)
except Exception as e:
    print("An unexpected error occurred:", e)

Enter the grade:  10
Enter the number of questions:  2
Enter the level of the questions:  medium
Enter the topic:  Cells of human bodu


# Agent: Passage generator and question framer
## Task: The generator agent is responsible for creating a descriptive educational passage of minimum 1 page for a specified grade level. This passage should be tailored to a difficulty level that aligns with the student's abilities. After generating the passage, the generator agent must also frame multiple-choice questions based on the passage content. These questions should be structured to assess the student's comprehension and critical thinking, with four answer choices per question. The agent must ensure that the passage and questions are contextually and grammatically accurate for the educational setting.


# Agent: Passage generator and question framer
## Thought: I need to generate a passage about the Cells of the human body suitable for grade 10 students and of medium difficulty level. Then, I need to create two multiple-choice questions based on the content of the passage, providing four answer choices for each question. In order

In [64]:
# Load questions from the JSON file
with open("questions.json", "r") as file:
    data = json.load(file)

responses = []
print(f"Passage: {data['passage']}")
for i, question in enumerate(data['questions'], start=1):
    print(f"\nQuestion {i}: {question['question']}")
    for j, option in enumerate(question['options'], start=1):
        print(f"{j}. {option}")
    response = int(input(f"Your answer (1-{len(question['options'])}): ").strip()) - 1
    responses.append(response)

# Save student responses to a JSON file
student_responses = {"responses": responses}
with open("responses.json", "w") as file:
    json.dump(student_responses, file, indent=4)


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Load questions and responses
with open("questions.json", "r") as q_file:
    questions_data = json.load(q_file)

with open("responses.json", "r") as r_file:
    responses_data = json.load(r_file)

# Evaluate student responses
evaluation_result = main_crew.kickoff(inputs={
    "questions": questions_data,
    "responses": responses_data
})

# Display evaluation feedback and scores
evaluation_output = evaluation_result.json()
print("\nEvaluation Report:")
print(f"Score: {evaluation_output['score']}%")
print("Feedback:")
for feedback in evaluation_output['feedback']:
    print(feedback)
